In [8]:
from utils import *
%matplotlib inline

In [10]:
st = "2020-07-19 00:00:00"
et = "2020-08-02 23:59:59"

In [16]:
power_df = get_data_section(client, '_power', st, et, uuid_dict, '1T')
power_df = power_df.tz_convert(tz_local).tz_localize(None).resample('5T').mean().interpolate('linear')
power_df.head()

,building,freezer,ref_comp,ref_fan,hvac_west,hvac_east
2020-07-19 00:00:00,35365.027344,4010.863464,1077.316412,343.767378,2930.059265,3236.995789
2020-07-19 00:05:00,35268.967773,3953.753296,1067.150353,289.990203,2914.825012,3223.700989
2020-07-19 00:10:00,37096.108594,3903.045020,860.188465,322.917995,2903.984229,3218.719873
2020-07-19 00:15:00,35680.070312,3861.996948,2004.883410,252.183642,2903.149536,3214.633057
2020-07-19 00:20:00,36675.044922,3836.057068,1530.843673,243.713170,2892.777100,3215.152466


In [17]:
weather_df = get_data_section(client, '_current', st, et, uuid_dict, '1T')
weather_df = weather_df.tz_convert(tz_local).tz_localize(None).resample('5T').mean().interpolate('linear')#.resample('15T').mean()
weather_df.head()

,oat,humidity,windspeed,cloudcover,solar,sr
2020-07-19 00:00:00,60.720000,76.000000,1.900000,0.02,0.0,0.0
2020-07-19 00:05:00,60.693333,76.000000,1.916667,0.02,0.0,0.0
2020-07-19 00:10:00,60.666667,76.000000,1.933333,0.02,0.0,0.0
2020-07-19 00:15:00,60.640000,76.000000,1.950000,0.02,0.0,0.0
2020-07-19 00:20:00,60.623333,76.333333,1.976667,0.02,0.0,0.0


In [65]:
power_df['solar'] = weather_df.sr * 272.6 * 0.26 * 0.98
battery_df = power_df[['building', 'solar']]
battery_df['building_pv'] = battery_df.building - battery_df.solar
battery_df['soc'] = 0.5
battery_df['battery_power'] = 0
battery_df = battery_df.dropna()

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [77]:
max_battery_rate = 21000
min_battery_rate = -21000
max_battery_soc = 0.95
min_battery_soc = 0.25
historical_data_interval_minutes = 15
battery_total_capacity = 40500

In [78]:
battery_df['time'] = (battery_df.index.hour + battery_df.index.minute/60 + battery_df.index.second/3600)
battery_df.time = round(battery_df.time, 2)
battery_df.head()

,building,solar,building_pv,soc,battery_power,time
2020-07-19 00:00:00,35365.027344,0.0,35365.027344,0.500000,3037.500000,0.00
2020-07-19 00:05:00,35268.967773,0.0,35268.967773,0.506250,3035.789696,0.08
2020-07-19 00:10:00,37096.108594,0.0,37096.108594,0.512496,3039.261154,0.17
2020-07-19 00:15:00,35680.070312,0.0,35680.070312,0.518750,3037.499263,0.25
2020-07-19 00:20:00,36675.044922,0.0,36675.044922,0.525000,3035.713549,0.33


In [82]:
prev_soc = -1
prev_setpoint = 0
for idx, row in battery_df.iterrows():
    if idx.time() == datetime.time(0,0,0):
        prev_soc = 0.5
        prev_setpoint = 0
    
    battery_soc = prev_soc  + prev_setpoint*5/60/battery_total_capacity
    battery_df.loc[idx, 'soc'] = battery_soc
    hour_now = row['time']
    net_load = row['building_pv']
    
    if idx.hour >= 0 and idx.hour <= 5:
        battery_setpoint = (max_battery_soc - battery_soc)*battery_total_capacity/(6-hour_now)
        if battery_setpoint > max_battery_rate:
            battery_setpoint = max_battery_rate
    elif net_load < 0:
        if battery_soc < max_battery_soc:
            if abs(net_load) > max_battery_rate:
                battery_setpoint = max_battery_rate
            else:
                battery_setpoint = abs(net_load)
        else:
            battery_setpoint = 0
    elif net_load > 0:
        if battery_soc > min_battery_soc:
            if (-1*net_load) < min_battery_rate:
                battery_setpoint = min_battery_rate
            else:
                battery_setpoint = -1*net_load
        else:
            battery_setpoint = 0

    battery_df.loc[idx, 'battery_power'] = battery_setpoint 
    prev_setpoint = battery_setpoint
    prev_soc = battery_soc

In [84]:
battery_df = battery_df[['soc', 'battery_power']]